### Imports

In [ ]:
from bs4 import BeautifulSoup
from itertools import combinations, chain
from datetime import datetime, timedelta
import config
import numpy as np

### Read Data

In [ ]:
print('reading matches')
with open(config.FILE, mode='r', encoding='utf-8') as f:
    soup = BeautifulSoup(f.read(), 'html.parser')

print('done')

#### OOP-ify data

In [ ]:

class ScoreboardEntry:
    def __init__(self, uname: str, uid: str, ping: int, kills: int, assists: int, deaths: int, mvps: int, hsp: int, score: int):
        self.uname = uname
        self.uid = uid
        self.ping = ping
        self.kills = kills
        self.assists = assists
        self.deaths = deaths
        self.mvps = mvps
        self.hsp = hsp
        self.score = score

class Scoreboard:
    START_CT = 2
    START_T = 3

    # t1 = ct starters, t2 = t starters
    def __init__(self, t1_rounds: int, t2_rounds: int, t1_entries: list, t2_entries: list):
        t1_info = {entry.uid: entry for entry in t1_entries}
        t2_info = {entry.uid: entry for entry in t2_entries}
        
        if config.USER in t1_info:
            self.friendly_stats = t1_info
            self.enemy_stats = t2_info

            self.rounds_won = t1_rounds
            self.rounds_lost = t2_rounds

            self.start_side = Scoreboard.START_CT
        else:
            self.friendly_stats = t2_info
            self.enemy_stats = t1_info

            self.rounds_won = t2_rounds
            self.rounds_lost = t1_rounds

            self.start_side = Scoreboard.START_T

        assert config.USER in self.friendly_stats and config.USER not in self.enemy_stats
        assert len(self.friendly_stats) == 5
        assert len(self.enemy_stats) == 5

class Game:
    WIN = 1
    TIE = 0
    LOSS = -1

    def __init__(self, map: str, date: datetime, queuetime: datetime, duration: timedelta, scoreboard: Scoreboard):
        self.map = map
        self.date = date
        self.duration = duration
        self.queuetime = queuetime
        self.scoreboard = scoreboard
        
        self.outcome = Game.WIN
        rw, rl = self.scoreboard.rounds_won, self.scoreboard.rounds_lost
        if rw == rl:
            self.outcome = Game.TIE
        elif rw < rl:
            self.outcome = Game.LOSS
    
    def __hash__(self) -> int:
        return self.date.__hash__()
    
    def __eq__(self, __value: object) -> bool:
        return type(__value) == Game and self.date == __value.date
    
    def __ne__(self, __value: object) -> bool:
        return type(__value) != Game or self.date != __value.date

#### Parse Data

In [ ]:
# parse data
NBSP = u'\xa0' # non-breaking space character

def parse_player(row) -> tuple:
    profile_element = row.td.find('div', class_='playerNickname ellipsis').a
    uname = profile_element.contents[0]
    profile_link = profile_element['href']
    uid = profile_link[str.rfind(profile_link, '/')+1:]
    return uname, uid

def parse_entry(row) -> ScoreboardEntry:
    tds = row.find_all('td')
    uname, uid = parse_player(row)

    # ping, kills, assists, deaths
    p, k, a, d = [int(x.contents[0]) for x in tds[1:5]]

    # mvps
    mvp_str = tds[5].contents[0]
    if mvp_str == NBSP:
        mvps = 0
    elif mvp_str == '★':
        mvps = 1
    else:
        mvps = int(mvp_str[1:])
    
    # hsp
    hsp_str = tds[6].contents[0]
    if hsp_str == NBSP:
        hsp = 0
    else:
        hsp = int(hsp_str[:-1])

    # score
    score = int(tds[7].contents[0])

    return ScoreboardEntry(uname=uname, uid=uid, ping=p, kills=k, assists=a, deaths=d, mvps=mvps, hsp=hsp, score=score)

def parse_duration(duration: str) -> timedelta:
    colon_count = duration.count(':')
    assert colon_count >= 1
    if colon_count == 1:
        dur = datetime.strptime(duration, '%M:%S')
        timedelta()
        return timedelta(minutes=dur.minute, seconds=dur.second)
    elif colon_count == 2:
        dur = datetime.strptime(duration, '%H:%M:%S')
        return timedelta(hours=dur.hour, minutes=dur.minute, seconds=dur.second)
    raise Exception('Failed to parse duration')

print('parsing matches')
matches = soup.find('table', class_='generic_kv_table csgo_scoreboard_root')
games = matches.tbody.find_all('tr', attrs={'style': 'display: table-row;'}, recursive=False)

all_parsed_games = []

for game in games:
    match_details, scoreboard_html = game.find_all('td', recursive=False)
    match_details = match_details.find('table', class_='csgo_scoreboard_inner_left')
    scoreboard_html = scoreboard_html.find('table', class_='csgo_scoreboard_inner_right')

    map, date, _, wait, duration = [str.strip(x.td.contents[0]) for x in match_details.tbody.find_all('tr')[:5]]
    wait = parse_duration(wait[len('Wait Time: '):])
    duration = parse_duration(duration[len('Match Duration: '):])

    scoreboard_rows = scoreboard_html.tbody.find_all('tr', recursive=False)
    board_1 = [parse_entry(x) for x in scoreboard_rows[1:6]]
    t1_score, t2_score = [int(str.strip(x)) for x in str.split(scoreboard_rows[6].td.contents[0], ':')]
    board_2 = [parse_entry(x) for x in scoreboard_rows[7:]]

    game_board = Scoreboard(t1_rounds=t1_score, t2_rounds=t2_score, t1_entries=board_1, t2_entries=board_2)
    all_parsed_games.append(Game(map=map, date=date, queuetime=wait, duration=duration, scoreboard=game_board))

n = len(all_parsed_games)
print(f'done. found {n} matches')

#### Identify Common Friends

In [ ]:
# map uid to uname
USER_NAMES = {}

# map uid to frequency
user_freq = {}

for game in all_parsed_games:
    for entry in game.scoreboard.friendly_stats.values():
        USER_NAMES[entry.uid] = entry.uname
        user_freq[entry.uid] = user_freq.get(entry.uid, 0) + 1
assert len(user_freq) > 0

GAME_THRESHOLD = 20

freq_filtered_friends = dict(filter(lambda item: item[1] >= GAME_THRESHOLD or USER_NAMES[item[0]] in config.keep_regardless, user_freq.items()))
freq_filtered_friends.pop(config.USER)

# map user name to uid for users that appear in at least 20 games
COMMON_FRIENDS = {USER_NAMES[uid]: uid for uid in freq_filtered_friends.keys()}

print(f'identified friends:')
for friend in COMMON_FRIENDS:
    print(f'{friend} ({COMMON_FRIENDS[friend]}): {user_freq[COMMON_FRIENDS[friend]]}')

#### Game Filtering Utilities

In [ ]:
all_maps = set(game.map for game in games)

ACTIVE_DUTY_MAPS = {'Competitive Nuke', 'Competitive Vertigo', 'Competitive Dust II', 'Competitive Overpass', 'Competitive Cache', 'Competitive Mirage', 'Competitive Ancient', 'Competitive Cobblestone', 'Competitive Inferno', 'Competitive Anubis', 'Competitive Train'}
other_maps = all_maps.difference(ACTIVE_DUTY_MAPS)

# ** Filtering utility predicates **

PREDICATE_REGISTRY = {}

# logical AND
def conjunction(*predicates):
    subreg = PREDICATE_REGISTRY.get(conjunction, {})
    if predicates in subreg:
        return PREDICATE_REGISTRY[conjunction][predicates]
    
    def tmp(game: Game):
        res = True
        [res := res and pred(game) for pred in predicates]
        return res
    tmp.__name__ = ", ".join([p.__name__ for p in predicates])
    subreg[predicates] = tmp
    PREDICATE_REGISTRY[conjunction] = subreg
    return tmp

# logical OR
def disjunction(*predicates):
    predicate_set = tuple(predicates)
    subreg = PREDICATE_REGISTRY.get(disjunction, {})
    if predicate_set in subreg:
        return PREDICATE_REGISTRY[disjunction][predicate_set]
    
    def tmp(game: Game):
        res = False
        [res := res or pred(game) for pred in predicate_set]
        return res
    tmp.__name__ = " OR ".join([p.__name__ for p in predicate_set])
    subreg[predicate_set] = tmp
    PREDICATE_REGISTRY[disjunction] = subreg
    return tmp

# logical NOT
def neg(pred):
    subreg = PREDICATE_REGISTRY.get(neg, {})
    if pred in subreg:
        return PREDICATE_REGISTRY[neg][pred]
    
    def tmp(game: Game):
        return not pred(game)
    tmp.__name__ = "NOT " + pred.__name__
    subreg[pred] = tmp
    PREDICATE_REGISTRY[neg] = subreg
    return tmp

# 0 ping on scoreboard
def has_disconnect(game: Game):
    return 0 in [player.ping for player in list(game.scoreboard.friendly_stats.values()) + list(game.scoreboard.enemy_stats.values())]

# returns function that returns true when all players in friendly_players are in the friendly board
def team_includes(friendly_players):
    if isinstance(friendly_players, str):
        friendly_players = frozenset((friendly_players,))
    else:
        friendly_players = frozenset(friendly_players)
    
    subreg = PREDICATE_REGISTRY.get(team_includes, {})
    if friendly_players in subreg:
        return PREDICATE_REGISTRY[team_includes][friendly_players]

    def tmp(game: Game):
        return friendly_players.issubset(game.scoreboard.friendly_stats.keys())
    
    friend_names = [USER_NAMES[uid] for uid in friendly_players]
    friend_names.sort()
    if len(friendly_players) == 0:
        tmp.__name__ = "team has anyone"
    else:
        tmp.__name__ = f"team has {', '.join(friend_names)}"
    subreg[friendly_players] = tmp
    PREDICATE_REGISTRY[team_includes] = subreg
    return tmp

# returns function that returns true when none of the players in friendly_players are in the friendly board
def team_excludes(friendly_players):
    if isinstance(friendly_players, str):
        friendly_players = frozenset((friendly_players,))
    else:
        friendly_players = frozenset(friendly_players)

    subreg = PREDICATE_REGISTRY.get(team_excludes, {})
    if friendly_players in subreg:
        return PREDICATE_REGISTRY[team_excludes][friendly_players]
    
    def tmp(game: Game):
        return friendly_players.isdisjoint(game.scoreboard.friendly_stats.keys())
        
    friend_names = [USER_NAMES[uid] for uid in friendly_players]
    friend_names.sort()
    tmp.__name__ = f"team excludes {', '.join(friend_names)}"
    subreg[friendly_players] = tmp
    PREDICATE_REGISTRY[team_excludes] = subreg
    return tmp

# returns true for active duty map (current or former)
def active_duty(game: Game):
    return game.map in ACTIVE_DUTY_MAPS

# games played on a particular map
def on_map(map: str):
    subreg = PREDICATE_REGISTRY.get(on_map, {})
    if map in subreg:
        return PREDICATE_REGISTRY[on_map][map]
    
    def tmp(game: Game):
        return game.map == map
    tmp.__name__ = f"on {map}"
    subreg[map] = tmp
    PREDICATE_REGISTRY[on_map] = subreg
    return tmp

# game won
def win(game: Game):
    return game.outcome == Game.WIN

# game tied
def tie(game: Game):
    return game.outcome == Game.TIE

# game lost
def loss(game: Game):
    return game.outcome == Game.LOSS

# returns true for long matches of score 16:X, X:16, or 15:15
def is_full_long_match(game: Game):
    rw, rl = game.scoreboard.rounds_won, game.scoreboard.rounds_lost
    return (rw == 16) or (rl == 16) or (rw == 15 and rl == 15)

# start ct
def start_ct(game: Game):
    return game.scoreboard.start_side == Scoreboard.START_CT

# start t
def start_t(game: Game):
    return game.scoreboard.start_side == Scoreboard.START_T

class GameFilter:
    def __init__(self, games: list) -> None:
        self.games = games
        self.filters = []
    
    def q(self, pred):
        remaining_games = list(filter(pred, self.games))
        filtered = GameFilter(remaining_games)
        filtered.filters = self.filters + [pred]
        return filtered
    
    def __len__(self):
        return len(self.games)
    
    def __iter__(self):
        return self.games.__iter__()


### Clean Data

In [ ]:
ALL_GAMES = GameFilter(all_parsed_games)

# remove games with disconnects [I'm actually leaving this off because I think people who DC at the end of the game count]
remove_dc = False
CLEANED = ALL_GAMES
if remove_dc:
    no_dc = ALL_GAMES.q(neg(has_disconnect))
    print(f"{len(no_dc)} games remaining without disconnects")
    CLEANED = no_dc

# remove non active-duty
CLEANED = CLEANED.q(active_duty)

# filter maps that have at least n games played
def maps_with_at_least_n_games(n):
    subreg = PREDICATE_REGISTRY.get(maps_with_at_least_n_games, {})
    if n in subreg:
        return subreg[n]
    
    map_fil_reg = {}
    def tmp(game: Game):
        if game.map in map_fil_reg:
            return map_fil_reg[game.map]
        result = len(ALL_GAMES.q(on_map(game.map))) >= n
        map_fil_reg[game.map] = result
        return result
    tmp.__name__ = f"maps with over {n} plays"
    subreg[n] = tmp
    PREDICATE_REGISTRY[maps_with_at_least_n_games] = subreg
    return tmp

# remove games on maps with fewer than 30 plays
CLEANED = CLEANED.q(maps_with_at_least_n_games(30))

# remove short matches and games that end early
CLEANED = CLEANED.q(is_full_long_match)
print(len(CLEANED))
print(set(g.map for g in CLEANED.games))

for map in set(g.map for g in CLEANED.games):
    assert len(list(filter(on_map, ALL_GAMES.games))) >= 30

### OOP Stats

In [ ]:
class GameStats:
    def __init__(self, games: GameFilter):
        self.games = games
        self.n = len(games.games)
        assert self.n > 0

        self.won_games = games.q(win)
        self.drawed_games = games.q(tie)
        self.lost_games = games.q(loss)
        self.wins = len(self.won_games)
        self.win_rate = self.wins / self.n
        self.losses = len(self.lost_games)
        self.loss_rate = self.losses / self.n
        self.draws = len(self.drawed_games)
        self.draw_rate = self.draws / self.n
    
    def __str__(self) -> str:
        if len(self.games.filters) > 3 and tuple(self.games.filters[0:3]) == (active_duty, maps_with_at_least_n_games(30), is_full_long_match):
            query_msg = "cleaned, " + f"{', '.join(f.__name__ for f in self.games.filters[3:])}"
        else:
            query_msg = ", ".join(f.__name__ for f in self.games.filters)
        return f'Stats for {query_msg} ({self.n} games played):\n\tWinrate = {self.win_rate:.3f}, Lossrate = {self.loss_rate:.3f}, Drawrate = {self.draw_rate:.3f} ({self.wins}:{self.losses}:{self.draws})'

class UserStats:
    def __init__(self, games: GameFilter, friendly: str):
        self.uid = friendly
        self.games = games
        user_entries = [g.scoreboard.friendly_stats[friendly] for g in games]
        user_entries: list[ScoreboardEntry]

        self.n = len(games)
        self.outcomes = np.array([g.outcome for g in games])
        self.rounds_won = np.array([g.scoreboard.rounds_won for g in games])
        self.rounds_lost = np.array([g.scoreboard.rounds_lost for g in games])
        self.rounds = self.rounds_won + self.rounds_lost
        assert len(self.rounds) == len(self.rounds_won)

        self.kills = np.array([entry.kills for entry in user_entries])
        self.assists = np.array([entry.assists for entry in user_entries])
        self.deaths = np.array([entry.deaths for entry in user_entries])
        self.ping = np.array([entry.ping for entry in user_entries])
        self.hsp = np.array([entry.hsp for entry in user_entries])
        self.score = np.array([entry.score for entry in user_entries])
        self.mvps = np.array([entry.mvps for entry in user_entries])

        self.total_rounds_won = sum(self.rounds_won)
        self.total_rounds_lost = sum(self.rounds_lost)
        self.total_rounds = sum(self.rounds)
        self.total_kills = sum(self.kills)
        self.total_assists = sum(self.assists)
        self.total_deaths = sum(self.deaths)
        self.total_ping = sum(self.ping)
        self.total_hsp = sum(self.hsp)
        self.total_score = sum(self.score)
        self.total_mvps = sum(self.mvps)

        self.kd = self.kills / self.deaths
        self.ka = self.kills + 0.4*self.assists
        self.kad = self.ka / self.deaths
        self.kpr = self.kills / self.rounds
        self.apr = self.assists / self.rounds
        self.dpr = self.deaths / self.rounds
        self.kapr = self.ka / self.rounds
        self.mvppr = self.mvps / self.rounds
        self.avg_kpr = self.total_kills / self.total_rounds
        self.avg_apr = self.total_assists / self.total_rounds
        self.avg_kapr = (self.total_kills + self.total_assists) / self.total_rounds
        self.avg_dpr = self.total_deaths / self.total_rounds
        self.avg_mvppr = self.total_mvps / self.total_rounds
        
        self.kpr_outcome_correlation = np.corrcoef(self.kpr, self.outcomes)[0,1]
        self.kd_outcome_correlation = np.corrcoef(self.kd, self.outcomes)[0,1]
        self.kad_outcome_correlation = np.corrcoef(self.kad, self.outcomes)[0,1]
        self.kapr_outcome_correlation = np.corrcoef(self.kapr, self.outcomes)[0,1]

        self.kpr_var = np.var(self.kpr)
        self.apr_var = np.var(self.apr)
        self.kapr_var = np.var(self.kapr)
        self.dpr_var = np.var(self.dpr)
        

### Basic Overall Stats

In [ ]:
with open('results/overall_stats.txt', 'w') as f:
    stats = GameStats(ALL_GAMES)
    f.write(f'ALL stats ({len(ALL_GAMES)} games played)\n')
    f.write(f'{stats}\n\n')
    
    long_active_duty_matches = ALL_GAMES.q(conjunction(is_full_long_match, active_duty))
    stats = GameStats(long_active_duty_matches)
    f.write(f'active duty long-match stats ({len(long_active_duty_matches)} games played)\n')
    f.write(f'{stats}\n\n')

    f.write(f'{GameStats(CLEANED)}\n\n')
    
    map_results = []
    for m in set(g.map for g in long_active_duty_matches):
        games = long_active_duty_matches.q(on_map(m))
        map_results.append(GameStats(games))

    map_results.sort(key=lambda x: x.loss_rate)
    for map_stats in map_results:
        f.write(f'{map_stats}\n\n')

#### Best Squads

In [ ]:
# get the stats for all games with each group of friends in friend_sets
def get_friend_game_stats(friend_sets, threshold=0):
    results = []

    for friend_group in friend_sets:
        games_with_friends = CLEANED.q(team_includes(friend_group))
        
        if len(games_with_friends) < threshold:
            continue
        
        friend_names = [USER_NAMES[uid] for uid in friend_group]
        friend_names.sort()
        stats = GameStats(games_with_friends)
        results.append(stats)
    
    return results

REQUIRED_GAMES = [None, 0, 20, 20, 20]
MAX_RESULTS = [None, 1000000, 20, 20, 20]

# Calculates stats for all possible queue cores (you + either 1, 2, 3 or 4 friends)
with open('results/squad_stats.txt', 'w') as f:
    for i in [1, 2, 3, 4]:
        friend_groups = [combo for combo in combinations(COMMON_FRIENDS.values(), i)]
        results = get_friend_game_stats(friend_groups, REQUIRED_GAMES[i])
        results.sort(key=lambda x: x.win_rate - x.loss_rate, reverse=True)
        to_show = min(MAX_RESULTS[i], len(results))

        f.write('##########################\n\n')
        f.write(f'Stats for core size {i+1}\n')
        f.write('\n')
        if len(results) > to_show:
            half = to_show // 2
            f.write(f'Over {to_show} results. Showing top and bottom {half}\n')
            results = results[0:half] + results[-half:]
        
        for stats in results:
            f.write(f'{stats}\n\n')


In [ ]:
results = []

for friend_to_include in COMMON_FRIENDS.values():
    for friend_to_exclude in COMMON_FRIENDS.values():
        if friend_to_exclude == friend_to_include:
            continue
        
        relevant_games = CLEANED.q(conjunction(team_includes(friend_to_include),
                                               team_excludes(friend_to_exclude)))

        if len(relevant_games) < 20:
            continue

        stats = GameStats(relevant_games)
        results.append(stats)

results.sort(key=lambda g: g.win_rate - g.loss_rate, reverse=True)

with open('results/squad_diff_stats.txt', 'w') as f:
    for result in results:
        f.write(f"{result}\n\n")

            

In [ ]:
results = []

for num_include in range(1, 5):
    for num_exclude in range(1, 6):
        for include_group in combinations(COMMON_FRIENDS.values(), num_include):
            relevant_included = CLEANED.q(team_includes(include_group))
            if len(relevant_included) < 30:
                continue
            
            # set of players appearing in any relevant_included games
            rel_included_players = set()
            for g in relevant_included:
                rel_included_players.update(g.scoreboard.friendly_stats.keys())

            for exclude_group in combinations(set(COMMON_FRIENDS.values()).difference(include_group), num_exclude):
                if not set(exclude_group).issubset(rel_included_players):
                    continue

                relevant_games = relevant_included.q(team_excludes(exclude_group))
                if len(relevant_games) < 30:
                    continue
                results.append(GameStats(relevant_games))

results.sort(key=lambda g: g.win_rate - g.loss_rate, reverse=True)

with open('results/squad_diff_stats_multi.txt', 'w') as f:
    for result in results:
        f.write(f"{result}\n\n")

#### Net-Winrate Deltas

In [ ]:
deltas = []

def net_winrate_games(games: GameFilter) -> float:
    stats = GameStats(games)
    return stats.win_rate - stats.loss_rate

def net_winrate(stats: GameStats) -> float:
    return stats.win_rate - stats.loss_rate

for friend in COMMON_FRIENDS.values():
    games_with_friend = CLEANED.q(team_includes(friend))
    games_without_friend = CLEANED.q(team_excludes(friend))
    stats_with = GameStats(games_with_friend)
    stats_without = GameStats(games_without_friend)
    deltas.append((USER_NAMES[friend], net_winrate(stats_with) - net_winrate(stats_without), stats_with, stats_without))

deltas.sort(key=lambda x: x[1], reverse=True)
with open('results/deltas.txt', 'w') as f:
    for friend, delta, stats_with, stats_without in deltas:
        f.write(f'{friend}: {delta:.3f}\n')
        f.write(f'{stats_with}\n')
        f.write(f'{stats_without}\n\n')
        

In [ ]:
friends = []
best_deltas, worst_deltas = [], []

for friend in COMMON_FRIENDS.values():
    best = -10
    worst = 10
    best_delta_data, worst_delta_data = None, None
    for coresize in range(0, 4):
        for core in combinations(set(COMMON_FRIENDS.values()).difference({friend}), coresize):
            coreset = frozenset(core)
            games_with_core_plus_friend = CLEANED.q(team_includes(coreset.union({friend})))
            games_with_core_without_friend = CLEANED.q(conjunction(
                team_includes(coreset),
                team_excludes(friend)
            ))

            if len(games_with_core_plus_friend) < 30 or len(games_with_core_without_friend) < 30:
                continue

            stats_with = GameStats(games_with_core_plus_friend)
            stats_without = GameStats(games_with_core_without_friend)
            delta = net_winrate(stats_with) - net_winrate(stats_without)
            if delta > best:
                best = delta
                best_delta_data = stats_with, stats_without
            if delta < worst:
                worst = delta
                worst_delta_data = stats_with, stats_without
        
    if not (best == -10 or worst == 10):
        friends.append(USER_NAMES[friend])
        best_deltas.append((best, best_delta_data))
        worst_deltas.append((worst, worst_delta_data))

zipped = list(zip(best_deltas, worst_deltas, friends))
zipped.sort(key=lambda x: x[0][0] - x[1][0])
with open('results/deltas_adv.txt', 'w') as f:
    for elt in zipped:
        friend = elt[2]
        best, (best_with, best_without) = elt[0]
        worst, (worst_with, worst_without) = elt[1]

        f.write(f'{friend}: best: {best:.3f}, worst: {worst:.3f}, diff: {best-worst:.3f}\n')
        f.write(f'{best_with}\n')
        f.write(f'{best_without}\n\n')
        f.write(f'{worst_with}\n')
        f.write(f'{worst_without}\n\n\n')


### Performance Correlations and Player Consistency

In [ ]:
stats = []

for friend in chain([config.USER], COMMON_FRIENDS.values()):
    friend_games = CLEANED.q(team_includes(friend))
    stats.append((UserStats(games=friend_games, friendly=friend), UserStats(games=friend_games, friendly=config.USER)))

stats.sort(key=lambda x: x[1].kapr_outcome_correlation)

with open("results/perf_comparison.txt", "w") as f:
    def str_of_stat(stat: UserStats):
        name: str = USER_NAMES[stat.uid]
        kapr = stat.avg_kapr
        dpr = stat.avg_dpr
        kapr_corr = stat.kapr_outcome_correlation
        kapr_var = stat.kapr_var
        return f"{name.rjust(22)}:\t\tKAPR={kapr:.3f}   DPR={dpr:.3f}   VAR(KAPR)={kapr_var:.3f}   KAPR-Win Correlaton={kapr_corr:.3f}"


    f.write("User Performance Stats: KAPR = Kills + 0.4*Assists per round:\n")
    for fstat, ustat in stats:
        fstat: UserStats
        ustat: UserStats
        fname = USER_NAMES[fstat.uid]
        uname = USER_NAMES[config.USER]
        n = len(fstat.games)
        f.write(f"Games with {fname} (n={n})\n")
        f.write(f"{str_of_stat(fstat)}\n")
        f.write(f"{str_of_stat(ustat)}\n\n")



### Maximizing Winrate

In [ ]:
from decision_tree import DecisionTree

filter_funcs = []

for player in COMMON_FRIENDS.values():
    filter_funcs.append(team_includes(player))

for map in ACTIVE_DUTY_MAPS:
    filter_funcs.append(on_map(map))

dt = DecisionTree(data=CLEANED, binary_labeller=win, split_funcs=filter_funcs)
with open('results/decision_tree.txt', 'w', encoding='utf-8') as f:
    f.write(str(dt))



In [ ]:
filter_funcs = []

for player in COMMON_FRIENDS.values():
    filter_funcs.append(team_includes(player))
    filter_funcs.append(neg(team_includes(player)))

for map in ACTIVE_DUTY_MAPS:
    filter_funcs.append(on_map(map))
    filter_funcs.append(neg(on_map(map)))
    

filter_funcs_extended = filter_funcs.copy()

for or_size in range(2, 4):
    for combo in combinations(filter_funcs, or_size):
        filter_funcs_extended.append(disjunction(*combo))

dt = DecisionTree(data=CLEANED, binary_labeller=win, split_funcs=filter_funcs_extended)
with open('results/decision_tree_big.txt', 'w', encoding='utf-8') as f:
    f.write(str(dt))

In [ ]:
filter_combinations = [frozenset()]
print(len(ACTIVE_DUTY_MAPS) + len(COMMON_FRIENDS))
for map in ACTIVE_DUTY_MAPS:
    new_combos = []
    for combo in filter_combinations:
        if len(combo) > 5:
            continue
        new_combos.append(combo.union({on_map(map)}))
        new_combos.append(combo.union({neg(on_map(map))}))
    filter_combinations.extend(new_combos)

for player in COMMON_FRIENDS.values():
    new_combos = []
    for combo in filter_combinations:
        if len(combo) > 5:
            continue
        new_combos.append(combo.union({team_includes(player)}))
        new_combos.append(combo.union({team_excludes(player)}))
    filter_combinations.extend(new_combos)

print(f"there are {len(filter_combinations)} combinations")

cleaned_stats = GameStats(CLEANED)
best = cleaned_stats
worst = cleaned_stats
for combo in filter_combinations:
    games = CLEANED.q(conjunction(*combo))
    if len(games) < 50:
        continue

    stats = GameStats(games)
    if net_winrate(stats) > net_winrate(best):
        best = stats
    if net_winrate(stats) < net_winrate(worst):
        worst = stats

with open('results/best_net_winrate.txt', 'w') as f:
    f.write(f"best net winrate: {net_winrate(best)}\n")
    f.write(f"{best}\n\n")
    f.write(f"worst net winrate: {net_winrate(worst)}\n")
    f.write(f"{worst}")

### Redact Output

In [ ]:
import os

all_names = list(USER_NAMES.values())
if len(all_names) != len(set(all_names)):
    print("WARNING: DUPLICATE NAMES APPEAR")

for filename in os.listdir("./results/"):
    path = f"./results/{filename}"
    result = ""

    with open(path, "r") as f:
        result = f.read()
    
    uname: str = USER_NAMES[config.USER]
    result = result.replace(uname.rjust(22), "USER".rjust(22))
    result = result.replace(uname, "USER")

    for i, name in enumerate(COMMON_FRIENDS.keys()):
        name: str
        result = result.replace(name.rjust(22), f"Friend {i+1}".rjust(22))
        result = result.replace(name, f"Friend {i+1}")
    
    with open(path, "w") as f:
        f.write(result)